In [2]:
import pandas as pd
import geopandas as gp
import os

import numpy as np



In [3]:
recovery_yrs = [2013, 2014, 2015, 2016, 2017]

In [4]:
for year in recovery_yrs:
    raw_hmda_by_year = [f for f in os.listdir("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/raw_hmda/") 
                        if f[-3:] == "csv"
                        and f.split('_')[1] == str(year) +".csv"]
    dfs = []
    for file in raw_hmda_by_year:
        df = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/raw_hmda/"+file)
        dfs.append(df)
raw_hmda_recovery = pd.concat(dfs, axis=0)[["tract_to_msamd_income", "hud_median_family_income", 'applicant_income_000s', 'applicant_race_name_1',
    'census_tract_number', 'county_name','applicant_ethnicity_name']] #concatenates all the dataframes from that year together
raw_hmda_recovery   
    

/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (30,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (29,30,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  inte

,tract_to_msamd_income,hud_median_family_income,applicant_income_000s,applicant_race_name_1,census_tract_number,county_name,applicant_ethnicity_name
0,124.650002,113100.0,785.0,Asian,6055.00,San Mateo County,Not Hispanic or Latino
1,113.279999,113100.0,142.0,White,6140.00,San Mateo County,Not Hispanic or Latino
2,240.979996,113100.0,632.0,White,6057.00,San Mateo County,Not Hispanic or Latino
3,97.949997,113100.0,NaN,White,6075.00,San Mateo County,Not Hispanic or Latino
4,113.279999,113100.0,45.0,Asian,6140.00,San Mateo County,Not Hispanic or Latino
5,104.239998,113100.0,117.0,Asian,6029.00,San Mateo County,Not Hispanic or Latino
6,142.710007,113100.0,325.0,Asian,6092.01,San Mateo County,Not Hispanic or Latino
7,113.279999,113100.0,52.0,Asian,6140.00,San Mateo County,Not Hispanic or Latino
8,99.089996,113100.0,177.0,Asian,6038.02,San Mateo County,Not Hispanic or Latino
9,137.199997,113100.0,170.0,Asian,6089.00,San Mateo County,Not Hispanic or Latino


In [6]:
raw_hmda_recovery['applicant_race_name_1'].unique()

array(['Asian', 'White',
       'Information not provided by applicant in mail, Internet, or telephone application',
       'Not applicable', 'Native Hawaiian or Other Pacific Islander',
       'Black or African American', 'American Indian or Alaska Native'],
      dtype=object)

In [7]:
place_crosswalk = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/geocorr2014.csv')
place_crosswalk.head()
hmda_recovery_places = raw_hmda_recovery.merge(place_crosswalk, how='left', left_on='census_tract_number', right_on='Tract')
hmda_recovery_places['applicant_race_name_1'].unique()



array(['Asian', 'White',
       'Information not provided by applicant in mail, Internet, or telephone application',
       'Not applicable', 'Native Hawaiian or Other Pacific Islander',
       'Black or African American', 'American Indian or Alaska Native'],
      dtype=object)

In [8]:
def code_geoid(row):
    if (np.isnan(row['census_tract_number'])):
        return "missing"
    else: 
        state = '6'
        tract = int(row['census_tract_number']*10)
        if (row['county_name'] == 'San Mateo County'):
            county = '081'
        if (row['county_name'] == 'San Francisco County'):
            county = '075'
        if (row['county_name'] == 'Napa County'):
            county = '055'
        if (row['county_name'] == 'Santa Clara County'):
            county = '085'
        if (row['county_name'] == 'Sonoma County'):
            county = '097'
        if (row['county_name'] == 'Solano County'):
            county = '095'
        if (row['county_name'] == 'Marin County'):
            county = '041'
        if (row['county_name'] == 'Alameda County'):
            county = '001'
        if (row['county_name'] == 'Contra Costa County'):
            county = '013'
        geoid = str(state + county + str(tract))
        #print(state, county, str(tract))
        #print(type(state), type(county), type(tract))
        return geoid


In [9]:
def code_race(row):
    race_code = ''
    if (row['applicant_ethnicity_name'] == 'Hispanic or Latino'):
        race_code = 'Hispanic or Latino'
    elif (row['applicant_ethnicity_name'] == 'Not Hispanic or Latino'):
        if (row['applicant_race_name_1'] == 'White'):
            race_code = 'NH White'
        elif (row['applicant_race_name_1'] == 'Asian'):
            race_code = 'NH Asian'
        elif (row['applicant_race_name_1'] == 'Black or African American'):
            race_code = 'NH Black'
        elif (row['applicant_race_name_1'] == 'Not applicable'):
            race_code = 'NH White'
        elif (row['applicant_race_name_1'] == 'Information not provided by applicant in mail, Internet, or telephone application'):
            race_code = 'NH White'
        else:
            race_code = 'NH Other'
    elif ((row['applicant_ethnicity_name'] == 'Not applicable') or (row['applicant_ethnicity_name'] == 'Information not provided by applicant in mail, Internet, or telephone application')):
        if (row['applicant_race_name_1'] == 'White'):
            race_code = 'NH White'
        elif (row['applicant_race_name_1'] == 'Asian'):
            race_code = 'NH Asian'
        elif (row['applicant_race_name_1'] == 'Black or African American'):
            race_code = 'NH Black'
        elif (row['applicant_race_name_1'] == 'Not applicable'):
            race_code = 'missing'
        elif (row['applicant_race_name_1'] == 'Information not provided by applicant in mail, Internet, or telephone application'):
            race_code = 'missing'
        else:
            race_code = 'NH Other'
    
   
    return race_code
        
    

In [10]:
def tract_eligibility(row):
    if (row["tract_to_msamd_income"] < 80):
        return 1
    else:
        return 0

In [14]:
hmda_recovery_places["CRA eligible tract"] = hmda_recovery_places.apply(lambda item: tract_eligibility(item), axis=1)
hmda_recovery_places["Race Code"] = hmda_recovery_places.apply(lambda row: code_race(row), axis=1)
#hmda_recovery_places['applicant_race_name_1'][21]
hmda_recovery_places.head()

,tract_to_msamd_income,hud_median_family_income,applicant_income_000s,applicant_race_name_1,census_tract_number,county_name,applicant_ethnicity_name,County code,Tract,State code,Place code,State abbreviation,County name,Place name,Total housing units (2010),tract to placefp allocation factor,CRA eligible tract,Race Code
0,124.650002,113100.0,785.0,Asian,6055.0,San Mateo County,Not Hispanic or Latino,6081.0,6055.0,6.0,9066.0,CA,San Mateo CA,"Burlingame city, CA",2909.0,1,0,NH Asian
1,113.279999,113100.0,142.0,White,6140.0,San Mateo County,Not Hispanic or Latino,6081.0,6140.0,6.0,65028.0,CA,San Mateo CA,"San Bruno city, CA",1775.0,1,0,NH White
2,113.279999,113100.0,142.0,White,6140.0,San Mateo County,Not Hispanic or Latino,6081.0,6140.0,6.0,99999.0,CA,San Mateo CA,,0.0,0,0,NH White
3,240.979996,113100.0,632.0,White,6057.0,San Mateo County,Not Hispanic or Latino,6081.0,6057.0,6.0,33633.0,CA,San Mateo CA,"Highlands-Baywood Park CDP, CA",0.0,0,0,NH White
4,240.979996,113100.0,632.0,White,6057.0,San Mateo County,Not Hispanic or Latino,6081.0,6057.0,6.0,33798.0,CA,San Mateo CA,"Hillsborough town, CA",1880.0,0.999,0,NH White


In [12]:
def race_vs_place(df):
    nhw_lm = 0 
    nhw_mh = 0
    h_lm = 0
    h_mh = 0
    a_lm = 0
    a_mh = 0
    b_lm = 0
    b_mh = 0
    o_lm = 0
    o_mh = 0
    m_lm = 0
    m_mh = 0
    
    df_lol = []
    for city, city_df in df.groupby("Place name"):
        a_lm = len(city_df[(city_df['Race Code']== 'NH Asian') & (city_df["CRA eligible tract"]== 1)])/len(city_df)
        nhw_lm = len(city_df[(city_df['Race Code']== 'NH White') & (city_df["CRA eligible tract"]== 1)])/len(city_df)
        h_lm = len(city_df[(city_df['Race Code']== 'Hispanic or Latino') & (city_df["CRA eligible tract"]== 1)])/len(city_df)
        b_lm = len(city_df[(city_df['Race Code']== 'NH Black') & (city_df["CRA eligible tract"]== 1)])/len(city_df)
        o_lm = len(city_df[(city_df['Race Code']== 'NH Other') & (city_df["CRA eligible tract"]== 1)])/len(city_df)
        m_lm = len(city_df[(city_df['Race Code']== 'missing') & (city_df["CRA eligible tract"]== 1)])/len(city_df)

        a_mh = len(city_df[(city_df['Race Code']== 'NH Asian') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
        nhw_mh = len(city_df[(city_df['Race Code']== 'NH White') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
        h_mh = len(city_df[(city_df['Race Code']== 'Hispanic or Latino') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
        b_mh = len(city_df[(city_df['Race Code']== 'NH Black') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
        o_mh = len(city_df[(city_df['Race Code']== 'NH Other') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
        m_mh = len(city_df[(city_df['Race Code']== 'missing') & (city_df["CRA eligible tract"]== 0)])/len(city_df)
    
        results = [city, round(h_lm, 3), round(h_mh, 3), round(nhw_lm, 3), round(nhw_mh, 3), round(a_lm, 3), 
                   round(a_mh, 3), round(b_lm, 3), round(b_mh, 3), round(o_lm, 3), round(o_mh, 3), round(m_lm, 3),
                   round(m_mh, 3)]
        df_lol.append(results)
        print(sum(results[1:]))
    data = pd.DataFrame(df_lol, columns=["City Name",
                                         'hispanic, LM', 'hispanic, MH', 
                                         'non hispanic white, LM','non hispanic white, MH', "asian, LM", "asian, MH", "black, LM",
                                         "black, MH", "other, LM", "other , MH", 'missing race/ethnicity, LM', 'missing race/ethnicity, MH'])
    
    return data
    
    
    
       
results = race_vs_place(hmda_recovery_places)
    
    

1.0010000000000001
1.0
0.999
1.0
1.0
1.0000000000000002
0.9990000000000001
1.0
1.0010000000000001
1.0
1.0
1.0
0.9990000000000001
1.0
1.0000000000000002
0.999
1.0
0.999
1.0
0.9990000000000001
1.0
1.0
1.0
1.0010000000000001
1.0
1.0
1.0
1.0
1.0
0.999
1.001
1.0
1.0
1.0
1.0010000000000001
0.999
0.9989999999999999
1.002
1.0000000000000002
1.0010000000000001
1.0
1.0010000000000001
0.9990000000000001
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0010000000000001
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0
0.9990000000000001
1.0000000000000002
1.0
1.0
1.0
1.0
1.0010000000000001
0.9990000000000001
1.0
1.0000000000000002
1.0
0.9990000000000001
1.0010000000000001
1.0
0.998
1.0
1.0
0.9990000000000001
1.001
0.9999999999999999
0.999
1.0010000000000001
0.9989999999999999
1.0
1.0010000000000001
0.9979999999999999
1.0
1.0010000000000001
1.001
1.0
1.0010000000000001
1.0010000000000001
1.0010000000000001
1.0
0.9990000000000001
1.0
1.001
1.0
1.0000000000000002
1.001
0

In [13]:
results.head(40)

#results.to_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/race_vs_tract_recovery.csv')


,City Name,"hispanic, LM","hispanic, MH","non hispanic white, LM","non hispanic white, MH","asian, LM","asian, MH","black, LM","black, MH","other, LM","other , MH","missing race/ethnicity, LM","missing race/ethnicity, MH"
0,,0.030,0.063,0.062,0.377,0.027,0.207,0.006,0.022,0.003,0.009,0.033,0.162
1,"Acalanes Ridge CDP, CA",0.000,0.031,0.000,0.566,0.000,0.093,0.000,0.023,0.000,0.023,0.000,0.264
2,"Alameda city, CA",0.007,0.021,0.095,0.327,0.030,0.229,0.001,0.014,0.001,0.004,0.049,0.221
3,"Alamo CDP, CA",0.000,0.051,0.000,0.553,0.000,0.142,0.000,0.006,0.000,0.006,0.000,0.242
4,"Albany city, CA",0.000,0.016,0.000,0.323,0.000,0.354,0.000,0.016,0.000,0.005,0.000,0.286
5,"Alhambra Valley CDP, CA",0.000,0.064,0.000,0.528,0.000,0.133,0.000,0.048,0.000,0.010,0.000,0.217
6,"Allendale CDP, CA",0.000,0.116,0.000,0.481,0.000,0.126,0.000,0.058,0.000,0.041,0.000,0.177
7,"Alto CDP, CA",0.000,0.012,0.000,0.704,0.000,0.086,0.000,0.000,0.000,0.000,0.000,0.198
8,"Alum Rock CDP, CA",0.153,0.000,0.166,0.000,0.485,0.000,0.003,0.000,0.008,0.000,0.186,0.000
9,"American Canyon city, CA",0.000,0.231,0.000,0.231,0.000,0.244,0.000,0.029,0.000,0.042,0.000,0.223


In [169]:
for place, place_df in hmda_recovery_places.groupby("Place name"):
    print(place)
    print(place_df.columns)
    break

 
Index(['tract_to_msamd_income', 'hud_median_family_income',
       'applicant_income_000s', 'applicant_race_name_1', 'census_tract_number',
       'county_name', 'applicant_ethnicity_name', 'County code', 'Tract',
       'State code', 'Place code', 'State abbreviation', 'County name',
       'Place name', 'Total housing units (2010)',
       'tract to placefp allocation factor'],
      dtype='object')


In [169]:
for place, place_df in hmda_recovery_places.groupby("Place name"):
    print(place)
    print(place_df.columns)
    break

 
Index(['tract_to_msamd_income', 'hud_median_family_income',
       'applicant_income_000s', 'applicant_race_name_1', 'census_tract_number',
       'county_name', 'applicant_ethnicity_name', 'County code', 'Tract',
       'State code', 'Place code', 'State abbreviation', 'County name',
       'Place name', 'Total housing units (2010)',
       'tract to placefp allocation factor'],
      dtype='object')


In [178]:
# hmda_recovery_places["CRA eligible tract"] = hmda_recovery_places.apply(lambda item: tract_eligibility(item), axis=1)
# hmda_recovery_places["Race Code"] = hmda_recovery_places.apply(lambda row: code_race(row), axis=1)
# hmda_recovery_places["Geoid"] = hmda_recovery_places.apply(lambda row: code_geoid(row), axis=1)
# hmda_recovery_places.head()

,tract_to_msamd_income,hud_median_family_income,applicant_income_000s,applicant_race_name_1,census_tract_number,county_name,applicant_ethnicity_name,CRA eligible tract,Race Code,Geoid,County code,Tract,State code,Place code,State abbreviation,County name,Place name,Total housing units (2010),tract to placefp allocation factor
0,124.650002,113100.0,785.0,Asian,6055.0,San Mateo County,Not Hispanic or Latino,0,NH Asian,608160550,6081.0,6055.0,6.0,9066.0,CA,San Mateo CA,"Burlingame city, CA",2909.0,1
1,113.279999,113100.0,142.0,White,6140.0,San Mateo County,Not Hispanic or Latino,0,NH White,608161400,6081.0,6140.0,6.0,65028.0,CA,San Mateo CA,"San Bruno city, CA",1775.0,1
2,113.279999,113100.0,142.0,White,6140.0,San Mateo County,Not Hispanic or Latino,0,NH White,608161400,6081.0,6140.0,6.0,99999.0,CA,San Mateo CA,,0.0,0
3,240.979996,113100.0,632.0,White,6057.0,San Mateo County,Not Hispanic or Latino,0,NH White,608160570,6081.0,6057.0,6.0,33633.0,CA,San Mateo CA,"Highlands-Baywood Park CDP, CA",0.0,0
4,240.979996,113100.0,632.0,White,6057.0,San Mateo County,Not Hispanic or Latino,0,NH White,608160570,6081.0,6057.0,6.0,33798.0,CA,San Mateo CA,"Hillsborough town, CA",1880.0,0.999


In [186]:
for city, city_df in hmda_recovery_places.groupby("Place name"):
    a_lm = len(city_df[(city_df['Race Code']== 'NH Asian') & (city_df["CRA eligible tract"]== 1)])
    print(city, a_lm)
    




  1158
Acalanes Ridge CDP, CA 0
Alameda city, CA 28
Alamo CDP, CA 0
Albany city, CA 0
Alhambra Valley CDP, CA 0
Allendale CDP, CA 0
Alto CDP, CA 0
Alum Rock CDP, CA 187
American Canyon city, CA 0
Angwin CDP, CA 0
Antioch city, CA 172
Ashland CDP, CA 76
Atherton town, CA 0
Bay Point CDP, CA 58
Bayview CDP, CA 20
Belmont city, CA 0
Belvedere city, CA 0
Benicia city, CA 0
Berkeley city, CA 13
Bethel Island CDP, CA 15
Black Point-Green Point CDP, CA 0
Blackhawk CDP, CA 0
Bloomfield CDP, CA 0
Bodega Bay CDP, CA 0
Bodega CDP, CA 0
Bolinas CDP, CA 0
Boyes Hot Springs CDP, CA 1
Brentwood city, CA 10
Brisbane city, CA 0
Broadmoor CDP, CA 0
Burbank CDP, CA 73
Burlingame city, CA 0
Byron CDP, CA 0
Calistoga city, CA 0
Cambrian Park CDP, CA 0
Camino Tassajara CDP, CA 0
Campbell city, CA 35
Carmet CDP, CA 0
Castle Hill CDP, CA 0
Castro Valley CDP, CA 126
Cazadero CDP, CA 6
Cherryland CDP, CA 204
Clayton city, CA 0
Cloverdale city, CA 0
Clyde CDP, CA 0
Colma town, CA 74
Concord city, CA 108
Contra C